# Load translation voxel simulation data

In [1]:
%%time
%load_ext autoreload

from analysis_utils import *
from joblib import Parallel, delayed
import multiprocessing  
import numpy as np
import pandas as pd

#Growth rate = 3.0
#Computes the translation latencies from our model with simulations containing
#1 to 14 cognate ternary complexes in the voxel (14 to 42 not included because
#the probability of voxels with that number of cognates is ~0)
path = "/Users/Akshay/Documents/tRNAShuffle/data/"
ensmbl_latency_dict = dict()
rxndiff=dict()
scalingFactor = 200
NR_scaledrates = {'k1r':718*200/scalingFactor,'k2f':1475*200/scalingFactor,'k2r_nr':1120*200/scalingFactor,'k3_nr':6*200/scalingFactor,'k4':209*200/scalingFactor}


data = "191003_2140/"
for i in range(1,7):
    sim_i = computeTransportRxnTimes(path+data,100*(i-1),100*(i-1)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))

data = "191105_1738/"
for i in range(7,16):
    sim_i = computeTransportRxnTimes(path+data, 100*(i-1-6),100*(i-1-6)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))
    
    
#Experimentally obtained codon usage frequencies and tRNA abundances at growth rate = 2.5 dbl/hr
gr25_Codon = np.array([2.36, 1.26, 45.55, 34.17, 16.97, 57.86, 19.27, 33.74, 14.98, 22.31, 43.18, 7.67, 24.11, 24.87, 39.49, 11.81, 0.03, 0.63, 2.19, 9.31, 17.22, 55.01, 5.61, 29.21, 21.67, 0.52, 15.79, 43.86, 4.17, 2.61, 20.64, 26.7, 7.03, 0.19, 2.76, 3.81, 6.72, 16.52, 4.27, 2.73, 7.92, 23.25, 2.51, 1.98, 16.33, 11.68, 0.62, 0.67, 43.82, 20.59, 27.28, 7.01, 6.78, 14.21, 60.75, 0.82, 3.86, 4.09, 28.82, 5.18, 4.38, 1.09])
gr25_pCodon = gr25_Codon/np.sum(gr25_Codon)
gr25_ptRNA = [0.0602409638554217, 0.010542168674698798, 0.07379518072289158, 0.00602409638554217, 0.010542168674698798, 0.00602409638554217, 0.021084337349397596, 0.043674698795180725, 0.021084337349397596, 0.01204819277108434, 0.01807228915662651, 0.08433734939759038, 0.03162650602409639, 0.07228915662650603, 0.01204819277108434, 0.07078313253012049, 0.06325301204819278, 0.01656626506024097, 0.009036144578313254, 0.027108433734939763, 0.010542168674698798, 0.03012048192771085, 0.013554216867469882, 0.015060240963855425, 0.0075301204819277125, 0.010542168674698798, 0.0075301204819277125, 0.003012048192771085, 0.021084337349397596, 0.004518072289156627, 0.01656626506024097, 0.01204819277108434, 0.0015060240963855425, 0.009036144578313254, 0.01656626506024097, 0.019578313253012052, 0.015060240963855425, 0.027108434, 0.05873493975903616, 0.019578313]
gr25_ptRNA = gr25_ptRNA/np.sum(gr25_ptRNA)

n_cores = 16

##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt)
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
Unique transcripts without a Gene identifier:  272
Sequences not available for # genes in Ecocyc DB:  153
Inconsistency in codons:  0
3767
Missing genes in transcriptome_dict in compute_transcript_distributions:  133
CPU times: user 2min 34s, sys: 1.46 s, total: 2min 35s
Wall time: 2min 36s


## minimal uniform random tRNA, minimal code GFP, 100 samples

In [2]:
%%time

gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),len(ptRNA_red20))
    ptRNA_uniform[tRNA_indices] = 0
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_GFP,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/minimaltRNA_minimalcode-sfGFP_100sample_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode-sfGFP_100sample_ptRNA',ptRNA_list)
print(np.average(elongt_list))

/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


466.7122760170306
CPU times: user 2.14 s, sys: 210 ms, total: 2.35 s
Wall time: 18 s


In [3]:
min(ptRNA_red20),max(ptRNA_red20)

(0.0, 0.12824427480759398)

In [4]:
gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')

minRange = min(ptRNA_red20[np.nonzero(ptRNA_red20)])
maxRange = max(ptRNA_red20)

a = ptRNA_list[np.argmin(elongt_list)]
print(a,sum(a))
print(computeElongationLatency(a,pCodon_GFP,ensmbl_latency_dict))
a = np.where((a>0)&(a<minRange), minRange, a)
a = np.where(a>maxRange, maxRange, a)
a_minmaxindices = np.where((a==minRange) | (a==maxRange))
a_indices = np.where((a>minRange) & (a<maxRange))
a[a_indices] = (a[a_indices]/sum(a[a_indices]))*(1-sum(a[a_minmaxindices]))
print(a,sum(a))
print(computeElongationLatency(a,pCodon_GFP,ensmbl_latency_dict))


[0.07882631 0.         0.12765575 0.         0.         0.
 0.05769538 0.10220107 0.01291003 0.         0.09904176 0.0981974
 0.12528648 0.         0.08588709 0.04660707 0.         0.
 0.09723007 0.         0.         0.07917451 0.01781629 0.0494213
 0.05502825 0.         0.         0.         0.0590446  0.
 0.         0.         0.10233595 0.         0.         0.
 0.03947795 0.06858593 0.         0.10788709] 1.5103102758120834
([417.2574203276397], [23.212012233002824])
[0.05219213 0.         0.08452286 0.         0.         0.
 0.03820101 0.06766893 0.00854793 0.         0.0655771  0.06501803
 0.08295413 0.         0.05686718 0.03085927 0.         0.
 0.06437754 0.         0.         0.05242268 0.01179644 0.03272261
 0.03643506 0.         0.         0.         0.03909435 0.
 0.         0.         0.06775823 0.         0.         0.
 0.02613896 0.04541181 0.         0.07143373] 0.9999999999999999
([422.15287180520977], [23.849071269689784])


## Genetic algorithm: faster tRNA distribution with minimal tRNA, GFP

In [5]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *
gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()
### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),objective='fast',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),np.average(elongt_list),'tRNA list: ',ptRNA_list[np.argmin(elongt_list)])
        
np.save('./data/gr06_UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/gr06_UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/gr06_UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Generation:  0  elongt:  421.36397558540443 462.6822894682783 tRNA list:  [0.07882631 0.         0.12765575 0.         0.         0.
 0.05769538 0.10220107 0.01291003 0.         0.09904176 0.0981974
 0.12528648 0.         0.08588709 0.04660707 0.         0.
 0.09723007 0.         0.         0.07917451 0.01781629 0.0494213
 0.05502825 0.         0.         0.         0.0590446  0.
 0.         0.         0.10233595 0.         0.         0.
 0.03947795 0.06858593 0.         0.10788709]
Generation:  100  elongt:  399.01813017052075 407.94268057112805 tRNA list:  [0.02871537 0.         0.01978458 0.         0.         0.
 0.04907707 0.09403572 0.01252991 0.         0.05464172 0.07364998
 0.08161851 0.         0.04625439 0.0453556  0.         0.
 0.07292446 0.         0.         0.08596478 0.01252991 0.04506478
 0.01260449 0.         0.         0.         0.03674125 0.
 0.         0.         0.06259239 0.         0.         0.
 0.02375287 0.0472468  0.         0.0949154 ]
Generation:  200  e

Generation:  1700  elongt:  389.2416182524307 394.7199245915154 tRNA list:  [0.02758598 0.         0.02906801 0.         0.         0.
 0.07745212 0.08020131 0.01253358 0.         0.02507193 0.07529217
 0.09171884 0.         0.04555441 0.05296026 0.         0.
 0.08451325 0.         0.         0.08105014 0.01253358 0.0438347
 0.01253358 0.         0.         0.         0.04194576 0.
 0.         0.         0.06705005 0.         0.         0.
 0.01307969 0.03960206 0.         0.08641858]
Generation:  1800  elongt:  388.8058768983608 394.1875714438587 tRNA list:  [0.02164523 0.         0.01491333 0.         0.         0.
 0.07562973 0.07624325 0.01221374 0.         0.04200595 0.06944695
 0.10269499 0.         0.03733073 0.05141847 0.         0.
 0.06937837 0.         0.         0.09739941 0.01221374 0.04307408
 0.01221374 0.         0.         0.         0.04934119 0.
 0.         0.         0.07823124 0.         0.         0.
 0.01221374 0.04722618 0.         0.07516596]
Generation:  1900

## Genetic algorithm: slower tRNA distribution with minimal tRNA, GFP

In [6]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),objective='slow',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', max(elongt_list),'tRNA list: ',ptRNA_list[np.argmax(elongt_list)])
        
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Generation:  0  elongt:  506.1319850512774 tRNA list:  [0.07175073 0.         0.11393193 0.         0.         0.
 0.01223108 0.02299913 0.04398516 0.         0.01904474 0.1003129
 0.01938072 0.         0.11923191 0.05015825 0.         0.
 0.06209129 0.         0.         0.04627819 0.11436115 0.05679264
 0.09855112 0.         0.         0.         0.0518254  0.
 0.         0.         0.0132013  0.         0.         0.
 0.04215812 0.04884841 0.         0.117174  ]
Generation:  100  elongt:  587.0551287231337 tRNA list:  [0.12117265 0.         0.08810009 0.         0.         0.
 0.03381009 0.01275431 0.0867927  0.         0.08187801 0.01275431
 0.01466565 0.         0.05178705 0.01497055 0.         0.
 0.01275431 0.         0.         0.01541613 0.12638762 0.03931565
 0.07525727 0.         0.         0.         0.01657615 0.
 0.         0.         0.02991053 0.         0.         0.
 0.10573831 0.04152895 0.         0.01842969]
Generation:  200  elongt:  603.6089489856116 tRNA list:  

Generation:  1800  elongt:  627.4355423613425 tRNA list:  [0.12824427 0.         0.02016024 0.         0.         0.
 0.020203   0.01458636 0.1213225  0.         0.12560355 0.01476579
 0.01554858 0.         0.01504514 0.01937392 0.         0.
 0.01534692 0.         0.         0.01557299 0.11949573 0.0204817
 0.12824427 0.         0.         0.         0.01995392 0.
 0.         0.         0.01563163 0.         0.         0.
 0.12749122 0.02207491 0.         0.02085333]
Generation:  1900  elongt:  627.4355423613425 tRNA list:  [0.12824427 0.         0.02016024 0.         0.         0.
 0.020203   0.01458636 0.1213225  0.         0.12560355 0.01476579
 0.01554858 0.         0.01504514 0.01937392 0.         0.
 0.01534692 0.         0.         0.01557299 0.11949573 0.0204817
 0.12824427 0.         0.         0.         0.01995392 0.
 0.         0.         0.01563163 0.         0.         0.
 0.12749122 0.02207491 0.         0.02085333]
CPU times: user 5min 2s, sys: 3.16 s, total: 5min 5s
W

In [7]:
print(min(ptRNA_red20[np.nonzero(ptRNA_red20)]))
print(max(ptRNA_red20))

0.012213740457866095
0.12824427480759398


## minimal uniform random tRNA, minimal code mRFP, 100 samples

In [43]:
%%time

gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/mRFP_RED20.csv')

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(ptRNA_red20),max(ptRNA_red20),len(ptRNA_red20))
    ptRNA_uniform[tRNA_indices] = 0
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_GFP,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/minimaltRNA_minimalcode-mRFP_100sample_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode-mRFP_100sample_ptRNA',ptRNA_list)
print(np.average(elongt_list))

184.3356414849228
CPU times: user 12.4 s, sys: 354 ms, total: 12.7 s
Wall time: 22 s


## Genetic algorithm: faster tRNA distribution with minimal tRNA, mRFP

In [44]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20),max(ptRNA_red20),objective='fast',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  168.74550105211244
Generation:  100  elongt:  160.17600220335808
Generation:  200  elongt:  156.65755391893552
Generation:  300  elongt:  155.83525207164303
Generation:  400  elongt:  155.79226660354004
Generation:  500  elongt:  155.31931077776082
Generation:  600  elongt:  154.47035931724673
Generation:  700  elongt:  154.47035931724673
Generation:  800  elongt:  154.34752486159513
Generation:  900  elongt:  154.34752486159513
Generation:  1000  elongt:  154.34752486159513
Generation:  1100  elongt:  153.24877589258833
Generation:  1200  elongt:  153.24877589258833
Generation:  1300  elongt:  153.24877589258833
Generation:  1400  elongt:  153.24877589258833
Generation:  1500  elongt:  153.24877589258833
Generation:  1600  elongt:  153.24877589258833
Generation:  1700  elongt:  153.24877589258833
Generation:  1800  elongt:  153.24877589258833
Generation:  1900  elongt:  153

## Genetic algorithm: slower tRNA distribution with minimal tRNA, mRFP

In [45]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20),max(ptRNA_red20),objective='slow',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  175.1247041405708
Generation:  100  elongt:  211.03438110718938
Generation:  200  elongt:  222.29216190704705
Generation:  300  elongt:  229.20293389041973
Generation:  400  elongt:  222.00841180441049
Generation:  500  elongt:  222.42975185073902
Generation:  600  elongt:  235.79605392618652
Generation:  700  elongt:  232.7473907291243
Generation:  800  elongt:  235.0525272570431
Generation:  900  elongt:  236.13680000893967
Generation:  1000  elongt:  238.68686851934845
Generation:  1100  elongt:  233.47878212946296
Generation:  1200  elongt:  233.2166650611879
Generation:  1300  elongt:  238.0814716424852
Generation:  1400  elongt:  235.79969613071896
Generation:  1500  elongt:  205.83571058026317
Generation:  1600  elongt:  235.51616418980637
Generation:  1700  elongt:  240.72484231501417
Generation:  1800  elongt:  240.78492448637914
Generation:  1900  elongt:  238.6056

In [9]:
import numpy as np
gr25_Codon = np.array([2.36, 1.26, 45.55, 34.17, 16.97, 57.86, 19.27, 33.74, 14.98, 22.31, 43.18, 7.67, 24.11, 24.87, 39.49, 11.81, 0.03, 0.63, 2.19, 9.31, 17.22, 55.01, 5.61, 29.21, 21.67, 0.52, 15.79, 43.86, 4.17, 2.61, 20.64, 26.7, 7.03, 0.19, 2.76, 3.81, 6.72, 16.52, 4.27, 2.73, 7.92, 23.25, 2.51, 1.98, 16.33, 11.68, 0.62, 0.67, 43.82, 20.59, 27.28, 7.01, 6.78, 14.21, 60.75, 0.82, 3.86, 4.09, 28.82, 5.18, 4.38, 1.09])
gr25_pCodon = gr25_Codon/np.sum(gr25_Codon)
gr25_ptRNA = [0.0602409638554217, 0.010542168674698798, 0.07379518072289158, 0.00602409638554217, 0.010542168674698798, 0.00602409638554217, 0.021084337349397596, 0.043674698795180725, 0.021084337349397596, 0.01204819277108434, 0.01807228915662651, 0.08433734939759038, 0.03162650602409639, 0.07228915662650603, 0.01204819277108434, 0.07078313253012049, 0.06325301204819278, 0.01656626506024097, 0.009036144578313254, 0.027108433734939763, 0.010542168674698798, 0.03012048192771085, 0.013554216867469882, 0.015060240963855425, 0.0075301204819277125, 0.010542168674698798, 0.0075301204819277125, 0.003012048192771085, 0.021084337349397596, 0.004518072289156627, 0.01656626506024097, 0.01204819277108434, 0.0015060240963855425, 0.009036144578313254, 0.01656626506024097, 0.019578313253012052, 0.015060240963855425, 0.027108434, 0.05873493975903616, 0.019578313]
gr25_ptRNA = gr25_ptRNA/np.sum(gr25_ptRNA)

max(gr25_ptRNA)

0.08523592085132133